In [ ]:
!pip install trulens-eval==0.19.1 llama-index==0.9.19 chromadb streamlit_javascript

In [ ]:
!pip freeze | grep pydantic

pydantic==2.5.2
pydantic_core==2.14.5


In [ ]:
!pip freeze | grep trulens-eval

trulens-eval==0.19.1


In [ ]:
!pip freeze | grep llama-index

llama-index==0.9.19


In [ ]:
from trulens_eval import Tru
tru = Tru()
tru.run_dashboard()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Starting dashboard ...
npx: installed 22 in 4.238s

Go to this url and submit the ip given here. your url is: https://legal-points-look.loca.lt

  Submit this IP Address: 34.74.141.85



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [2]:
from llama_index import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings import OpenAIEmbedding
from llama_index.llms import OpenAI
from llama_index.vector_stores import QdrantVectorStore
from llama_index import StorageContext

from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [3]:
import pathlib
import textwrap
import json
import time
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

project_id = userdata.get('PROJECT_ID')
!gcloud config set project {project_id}
# !gcloud auth application-default login

import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Updated property [core/project].


In [21]:
# Create a local Qdrant vector store
import chromadb
from llama_index.vector_stores import ChromaVectorStore

chroma_client = chromadb.EphemeralClient()
# chroma_collection = chroma_client.create_collection("chroma_store")
vector_store = ChromaVectorStore(
    chroma_collection="chroma_store",
)


# Using the embedding model to Gemini
embed_model = OpenAIEmbedding(
    model_name="text-embedding-ada-002", api_key=OPENAI_API_KEY
)

service_context = ServiceContext.from_defaults(
    llm=OpenAI(api_key=OPENAI_API_KEY), embed_model=embed_model
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader("/content/data").load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

index.storage_context.persist()

In [22]:

print(chroma_collection.get())

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': [], 'uris': None, 'data': None}


In [ ]:
!zip -r storage.zip /content/storage/

  adding: content/storage/ (stored 0%)
  adding: content/storage/default__vector_store.json (deflated 60%)
  adding: content/storage/index_store.json (deflated 49%)
  adding: content/storage/graph_store.json (stored 0%)
  adding: content/storage/docstore.json (deflated 70%)
  adding: content/storage/image__vector_store.json (deflated 19%)


In [6]:
from pydantic import BaseModel

class InsaProfile(BaseModel):
    """Data model for a InsaProfile."""
    handle: str
    name: str
    email: str
    bio: str
    profile_image_url: str

In [16]:
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='/content/storage')
index = load_index_from_storage(storage_context, service_context = service_context)

query_engine = index.as_query_engine(output_cls=InsaProfile, response_mode="compact")
response = query_engine.query("give me all index details")
# print(f"Response: {response}")
print(response.response)
print(type(response.response))

handle='sincerelyjules' name='JULIE SARIÑANA' email='jen@jenniferpowell.com' bio='california girl dreaming big. ✿ \nfounder of @colordept\nagent: jen@jenniferpowell.com\nshop my line at TjMaxx, Marshall’s + Homegoods' profile_image_url=''
<class '__main__.InsaProfile'>


In [ ]:
#RAG using lamaindex and evaluate using trulens

from trulens_eval import Feedback
from trulens_eval.feedback.provider import OpenAI
# from trulens_eval.feedback.provider import Huggingface


# Initialize provider class
provider = OpenAI()
# hugs_provider = Huggingface()

# LLM-based feedback functions
f_controversiality = Feedback(
    provider.controversiality_with_cot_reasons,
    name="Controversiality",
    higher_is_better=False,
    ).on_output()

f_criminality = Feedback(
    provider.criminality_with_cot_reasons,
    name="Criminality",
    higher_is_better=False,
    ).on_output()

f_insensitivity = Feedback(
    provider.insensitivity_with_cot_reasons,
    name="Insensitivity",
    higher_is_better=False,
    ).on_output()

f_maliciousness = Feedback(
    provider.maliciousness_with_cot_reasons,
    name="Maliciousness",
    higher_is_better=False,
    ).on_output()

# Moderation feedback functions
f_hate = Feedback(
    provider.moderation_hate,
    name="Hate",
    higher_is_better=False
    ).on_output()

f_hatethreatening = Feedback(
    provider.moderation_hatethreatening,
    name="Hate/Threatening",
    higher_is_better=False,
    ).on_output()

f_violent = Feedback(
    provider.moderation_violence,
    name="Violent",
    higher_is_better=False
    ).on_output()

f_violentgraphic = Feedback(
    provider.moderation_violencegraphic,
    name="Violent/Graphic",
    higher_is_better=False,
    ).on_output()

f_selfharm = Feedback(
    provider.moderation_selfharm,
    name="Self Harm",
    higher_is_better=False
    ).on_output()

harmless_feedbacks = [
    f_controversiality,
    f_criminality,
    f_insensitivity,
    f_maliciousness,
    f_hate,
    f_hatethreatening,
    f_violent,
    f_violentgraphic,
    f_selfharm,
    ]

query_engine = index.as_query_engine(
    similarity_top_k=1,
)

from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(
        query_engine,
        app_id='Multi-index query harmless feedback',
        feedbacks=harmless_feedbacks
    )
with tru_query_engine_recorder as recording:
    response = query_engine.query("What did the author do growing up?")

print(response)